## 네이버 뉴스 본문 수집

- 네이버뉴스 링크의 도메인에 따라 본문 수집 코드를 다르게 작성해야 함
  - `https://n.news`로 시작하는 뉴스 본문은 nlink로 HTTP 요청 실행
  - `https://m.sports`로 시작하는 뉴스 본문은 `https://api-gw.sports`로 변경해야 함
  - `https://m.entertain`으로 시작하는 뉴스 본문은 `https://api-gw.entertain`으로 변경해야 함

### 데이터 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BTS
import json
import re
import time
from tqdm.notebook import tqdm

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

In [ ]:
# pkl 파일을 읽고 newsLinks를 생성합니다.
newsLinks = pd.read_pickle(filepath_or_buffer = 'Naver_News_Link.pkl')

In [ ]:
# newsLinks에서 Domain의 도수를 확인합니다.
newsLinks.str.extract(pat = '(https://.+?(?=/article))').value_counts()

### 함수 생성

In [ ]:
# 네이버뉴스 링크로 HTTP 요청을 실행하고 뉴스 본문을 반환하는 함수를 생성합니다.
def NaverNewsBody(nlink):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열을 처리하는 코드를 분기합니다.
    if 'https://n.news' in nlink:
        
        # 네이버뉴스 링크로 HTTP 요청을 실행합니다.
        res = requests.get(url = nlink)
        
        # HTTP 응답 바디 문자열을 bs4.BeautifulSoup 객체로 변환합니다.
        soup = BTS(markup = res.text, features = 'html.parser')
        
        # soup에서 뉴스 본문을 포함하는 HTML 요소를 선택합니다.
        item = soup.select_one(selector = '#dic_area, div._article_content')
        
        # item의 텍스트를 반환합니다.
        # [참고] strip()은 문자열 양 옆에 있는 공백을 모두 제거합니다.
        return item.text.strip()
    
    elif 'https://m' in nlink:
        
        # 'https://m'으로 시작하는 nlink의 일부 문자열을 변경한 url을 생성합니다.
        url = re.sub(pattern = 'https://m.(sports|entertain).naver.com(.+)?(?=/article)', 
                     repl = r'https://api-gw.\1.naver.com/news', 
                     string = nlink)
        
        # url로 HTTP 요청을 실행합니다.
        res = requests.get(url = url)
        
        # JSON 형태의 문자열을 딕셔너리로 변환합니다.
        dat = json.loads(s = res.text)
        
        # 뉴스 본문을 문자열로 반환합니다.
        return dat['result']['articleInfo']['article']['content']

### 데이터 수집

In [ ]:
# 경고를 출력하지 않도록 설정합니다.
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
# 반복 실행할 횟수를 n에 할당합니다.
n = len(newsLinks)

# 뉴스 본문을 저장할 시리즈를 생성합니다.
newsTexts = pd.Series(data = [np.nan] * n, dtype = str)

# for 반복문으로 네이버뉴스 본문을 수집합니다.
for i in tqdm(range(n)):
    
    # 네이버뉴스 링크로 뉴스 본문을 수집하고 newsTexts에 i번째 원소로 할당합니다.
    newsTexts.iloc[i] = NaverNewsBody(nlink = newsLinks.iloc[i])
    
    # 1초간 멈춥니다.
    time.sleep(1)

In [ ]:
# newsTexts의 처음 5행을 확인합니다.
newsTexts.head()

### 데이터 전처리

In [ ]:
# newsTexts에서 title="한글" 패턴이 있는지 확인합니다.
newsTexts.str.contains(pat = '(title="[가-힣 ]+")').sum()

In [ ]:
# newsTexts에서 title="한글" 패턴을 추출하고 도수를 확인합니다.
newsTexts.str.extractall(pat = '(title="[가-힣 ]+")')[0].value_counts()

In [ ]:
# newsTexts에서 title="한글" 패턴을 삭제하고 newsTexts에 재할당합니다.
newsTexts = newsTexts.str.replace(pat = '(title="[가-힣 ]+")', repl = '', regex = True)

In [ ]:
# newsTexts에서 HTML 요소 문자열이 있는지 확인합니다.
# [참고] 뉴스 본문에서 제목 등을 강조하기 위해 꺽쇠 괄호로 묶는 경우가 있습니다.
newsTexts.str.contains(pat = '(<[^가-힣]+?>)').sum()

In [ ]:
# newsTexts에서 HTML 요소 문자열을 추출하고 도수를 확인합니다.
newsTexts.str.extractall(pat = '(<[^가-힣]+?>)')[0].value_counts()

In [ ]:
# newsTexts에서 HTML 요소 문자열을 삭제하고 newsTexts에 재할당합니다.
newsTexts = newsTexts.str.replace(pat = '(<[^가-힣]+?>)', repl = '', regex = True)

In [ ]:
# newsTexts의 처음 5행을 확인합니다.
newsTexts.head()

### 외부 파일로 저장

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# newsTexts를 pkl 파일로 저장합니다.
pd.to_pickle(obj = newsTexts, filepath_or_buffer = 'Naver_News_Text.pkl')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

## End of Document